## MDLE - Project 2

In [1]:
import pandas as pd

tracks = pd.read_csv("fma_metadata/tracks.csv", encoding='utf-8-sig')

tracks


/var/folders/g3/kj70w42d2kz14wq0r9tqx1yr0000gn/T/ipykernel_97447/224490643.py:3: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv("fma_metadata/tracks.csv", encoding='utf-8-sig')


,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106571,155316,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,122,NaN,Creative Commons Attribution-NonCommercial-NoD...,102,NaN,3,NaN,[],The Auger
106572,155317,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,194,NaN,Creative Commons Attribution-NonCommercial-NoD...,165,NaN,4,NaN,[],Let's Skin Ruby
106573,155318,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,214,NaN,Creative Commons Attribution-NonCommercial-NoD...,168,NaN,6,NaN,[],My House Smells Like Kim Deal/Pulp
106574,155319,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,336,NaN,Creative Commons Attribution-NonCommercial-NoD...,294,NaN,5,NaN,[],The Man With Two Mouths


In [2]:

# Get the first and second row
first_row = tracks.iloc[0, :]
second_row = tracks.iloc[1, :]

# Get the new column names from the first and second row
new_column_names = [second_row.iloc[0]] + list(first_row.iloc[1:].values)

# Rename the columns of the DataFrame
tracks.columns = new_column_names
tracks = tracks.iloc[2:, :].reset_index(drop=True)
tracks


,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106569,155316,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,122,NaN,Creative Commons Attribution-NonCommercial-NoD...,102,NaN,3,NaN,[],The Auger
106570,155317,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,194,NaN,Creative Commons Attribution-NonCommercial-NoD...,165,NaN,4,NaN,[],Let's Skin Ruby
106571,155318,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,214,NaN,Creative Commons Attribution-NonCommercial-NoD...,168,NaN,6,NaN,[],My House Smells Like Kim Deal/Pulp
106572,155319,0,2017-03-30 15:20:35,2017-02-17 00:00:00,NaN,0,22940,"<p>A live performance at Monty Hall on Feb 17,...",1506,Monty Hall,...,NaN,336,NaN,Creative Commons Attribution-NonCommercial-NoD...,294,NaN,5,NaN,[],The Man With Two Mouths


In [3]:
tracks = tracks.loc[tracks['subset'] == 'small'].reset_index(drop=True)
tracks

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
2,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
3,140,1,2008-11-26 01:49:59,2007-05-22 00:00:00,NaN,1,61,<p>Alec K. Redfearn &amp; The Eyesores: Ellen ...,1300,"Alec K. Refearn, Rob Pemberton",...,NaN,1593,en,Attribution-Noncommercial-No Derivative Works ...,1299,NaN,2,NaN,[],Queen Of The Wires
4,141,0,2008-11-26 01:49:57,2009-01-16 00:00:00,NaN,1,60,"<p>A full ensamble of strings, drums, electron...",1304,NaN,...,NaN,839,en,Attribution-Noncommercial-No Derivative Works ...,725,NaN,4,NaN,[],Ohio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,154308,0,2017-03-05 04:57:38,2017-03-05 00:00:00,NaN,0,22780,NaN,22334,Fleslit,...,NaN,3371,NaN,Attribution,2705,NaN,17,NaN,"['fleslit', 'trap beat free use', 'trap beat f...",MIA
7996,154309,0,2017-03-05 04:57:38,2017-03-05 00:00:00,NaN,0,22780,NaN,22334,Fleslit,...,NaN,4525,NaN,Attribution,3589,NaN,18,NaN,"['fleslit', 'trap beat free use', 'trap beat f...",A1 Symphony
7997,154413,0,2017-03-07 18:44:11,NaN,Ernie Indradat,0,22789,<p>A live performance at WFMU for Dark Night o...,3777,Julie Bennack,...,NaN,809,NaN,Creative Commons Attribution-NonCommercial-NoD...,676,NaN,9,NaN,[],Do Easy
7998,154414,0,2017-03-07 18:44:11,NaN,Ernie Indradat,0,22789,<p>A live performance at WFMU for Dark Night o...,3777,Julie Bennack,...,NaN,851,NaN,Creative Commons Attribution-NonCommercial-NoD...,788,NaN,10,NaN,[],Dead Can Dance (uncensored)


In [4]:
tracks.shape

(8000, 53)

In [5]:
from sklearn.cluster import AgglomerativeClustering


In [6]:
tracks.shape[0]

8000

In [7]:
features = pd.read_csv("fma_metadata/features.csv", encoding='utf-8-sig')
features

/var/folders/g3/kj70w42d2kz14wq0r9tqx1yr0000gn/T/ipykernel_97447/3656559579.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,2

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,statistics,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
1,number,01,02,03,04,05,06,07,08,09,...,04,05,06,01,01,01,01,01,01,01
2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
4,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106572,155316,-0.490129,0.463834,2.32197,-0.084352,1.662914,2.115189,-0.237794,5.695442,0.830353,...,0.12841,0.022547,0.019816,4.448255,0.172852,0.028773,0.02832,0.003906,0.955388,0.012385
106573,155317,-0.461559,-0.229601,-0.496632,-0.422033,0.130612,-0.263825,-0.628103,-0.082687,-0.229483,...,0.132964,0.023548,0.026527,3.270612,0.196289,0.031116,0.027832,0.002441,1.28306,0.019059
106574,155318,0.552473,-0.110498,-0.532014,0.263131,-0.224011,-0.530972,1.713526,1.418444,1.325197,...,0.108324,0.01754,0.020471,2.356727,0.212891,0.03845,0.037109,0.003418,0.828569,0.017904
106575,155319,-0.176901,0.187208,-0.050664,0.368843,0.066005,-0.857354,-0.78086,0.626281,-0.630938,...,0.088311,0.018328,0.017936,6.188604,0.16748,0.04148,0.038086,0.004883,1.81874,0.020133


In [8]:
first_row = features.iloc[0, :]
second_row = features.iloc[1, :]
third_row = features.iloc[2, :]

# Get the new column names from the first and second row
new_column_names = [third_row.iloc[0]] + list(first_row.iloc[1:].values)

# Rename the columns of the DataFrame
features.columns = new_column_names
features = features.iloc[3:, :].reset_index(drop=True)
features

,track_id,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
0,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
1,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
2,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
3,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,7.4357867241e-02,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02
4,20,-1.9383698702e-01,-1.9852678478e-01,2.0154602826e-01,2.5855624676e-01,7.7520370483e-01,8.4794059396e-02,-2.8929358721e-01,-8.1641042233e-01,4.3850939721e-02,...,9.5002755523e-02,2.2492416203e-02,2.1355332807e-02,1.6669036865e+01,4.6972656250e-01,4.7224905342e-02,4.0039062500e-02,9.7656250000e-04,3.1898307800e+00,3.0992921442e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106569,155316,-0.490129,0.463834,2.32197,-0.084352,1.662914,2.115189,-0.237794,5.695442,0.830353,...,0.12841,0.022547,0.019816,4.448255,0.172852,0.028773,0.02832,0.003906,0.955388,0.012385
106570,155317,-0.461559,-0.229601,-0.496632,-0.422033,0.130612,-0.263825,-0.628103,-0.082687,-0.229483,...,0.132964,0.023548,0.026527,3.270612,0.196289,0.031116,0.027832,0.002441,1.28306,0.019059
106571,155318,0.552473,-0.110498,-0.532014,0.263131,-0.224011,-0.530972,1.713526,1.418444,1.325197,...,0.108324,0.01754,0.020471,2.356727,0.212891,0.03845,0.037109,0.003418,0.828569,0.017904
106572,155319,-0.176901,0.187208,-0.050664,0.368843,0.066005,-0.857354,-0.78086,0.626281,-0.630938,...,0.088311,0.018328,0.017936,6.188604,0.16748,0.04148,0.038086,0.004883,1.81874,0.020133


In [9]:
ids = list(set(tracks.index.to_list()))
features_sm = features.loc[ids]
features_sm = features_sm.drop(['track_id'], axis=1)
features_sm

,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
0,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,1.3412306309e+00,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
1,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,6.1918509007e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
2,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,1.0385463238e+00,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
3,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,1.9823863506e+00,...,7.4357867241e-02,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02
4,-1.9383698702e-01,-1.9852678478e-01,2.0154602826e-01,2.5855624676e-01,7.7520370483e-01,8.4794059396e-02,-2.8929358721e-01,-8.1641042233e-01,4.3850939721e-02,-8.0476069450e-01,...,9.5002755523e-02,2.2492416203e-02,2.1355332807e-02,1.6669036865e+01,4.6972656250e-01,4.7224905342e-02,4.0039062500e-02,9.7656250000e-04,3.1898307800e+00,3.0992921442e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-0.452105,-0.947249,1.284448,-0.786291,-1.092085,0.004437,-0.897571,-0.82373,-0.590082,-1.446606,...,0.131945,0.025486,0.031835,46.542461,0.356445,0.024215,0.018555,0.005859,4.626769,0.018366
7996,-0.640964,-0.889433,-1.319779,-0.807146,0.605039,0.563072,-0.576786,-1.207992,-0.199673,-1.162841,...,0.144172,0.016904,0.032727,33.016399,0.378906,0.02745,0.020508,0.006348,4.839769,0.025371
7997,0.052395,-0.61241,-0.466664,-1.026164,-0.838505,-0.584648,0.365097,0.366401,0.291545,-0.690754,...,0.12638,0.033617,0.021355,21.049704,0.338379,0.035065,0.026855,0.009277,3.75108,0.026121
7998,-0.672998,-1.016979,-0.931607,-0.479711,-1.09837,0.555661,0.189147,-0.094367,0.550761,-0.572284,...,0.149132,0.025151,0.025916,22.643829,0.36377,0.046089,0.037598,0.004883,3.882393,0.032596


In [10]:
df_numeric = features_sm.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [11]:
df_numeric.shape

(8000, 518)

In [12]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances

def cluster_metrics(data, labels):
    clusters = np.unique(labels)
    radii = []
    diameters = []
    densities = []

    for cluster in clusters:
        cluster_data = data[labels == cluster]
        intra_distances = pairwise_distances(cluster_data)
        radius = np.max(np.min(intra_distances, axis=1))
        diameter = np.max(intra_distances)
        density = len(cluster_data) / (np.pi * (radius ** 2))

        radii.append(radius)
        diameters.append(diameter)
        densities.append(density)

    return radii, diameters, densities


In [14]:
results = []
for k in range(8, 17):
    model = AgglomerativeClustering(n_clusters=k)
    labels = model.fit_predict(df_numeric)
    
    radii, diameters, densities = cluster_metrics(df_numeric, labels)
    
    result = {
        'k': k,
        'radii': radii,
        'diameters': diameters,
        'densities': densities
    }
    results.append(result)

# Print results
for result in results:
    print(f"Number of clusters (k): {result['k']}")
    print(f"Radius: {result['radii']}")
    print(f"Diameters: {result['diameters']}")
    print(f"Densities: {result['densities']}")
    print()

Number of clusters (k): 8
Radius: [0.0006905339660024879, 0.0006459353786778786, 0.0009134905729428568, 0.0006459353786778786, 0.0008457279333832408, 0.0005459150335692846, 0.0006905339660024879, 0.0003860202221885229]
Diameters: [8225.768734539772, 4958.138646209162, 11883.765063581504, 8748.108816757504, 6545.349946766498, 6342.133564686588, 4421.581242929958, 7386.785214640227]
Densities: [1411188469.30525, 1554042931.1932936, 40434106.70262375, 571417847.5521733, 228745150.81138405, 418683731.2905643, 1294368231.779981, 318285081.4402759]

Number of clusters (k): 9
Radius: [0.0006905339660024879, 0.0006459353786778786, 0.0009134905729428568, 0.0006459353786778786, 0.0008457279333832408, 0.0005459150335692846, 0.0006905339660024879, 0.0003860202221885229, 0.00048828125]
Diameters: [6159.077251506113, 4958.138646209162, 11883.765063581504, 8748.108816757504, 6545.349946766498, 6342.133564686588, 4421.581242929958, 7386.785214640227, 3832.553092803292]
Densities: [1157521667.8218086, 

In [20]:
features_complete = features.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [3]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, MiniBatchKMeans
from sklearn.metrics import silhouette_score


# Define the range of k values to test
k_values = range(8, 17)


# Calculate the distance matrix
dist_matrix = pairwise_distances(df_numeric, metric="euclidean")

# Calculate the radius, diameter and density for each k value
for k in k_values:
    clustering = AgglomerativeClustering(n_clusters=k, affinity="precomputed", linkage="complete").fit(dist_matrix)
    labels = clustering.labels_
    cluster_centers = pd.DataFrame(df_numeric).groupby(labels).mean()
    radius = pd.DataFrame(df_numeric).groupby(labels).apply(lambda x: pd.Series(x.apply(lambda x: np.linalg.norm(x - cluster_centers.loc[x.name]), axis=1).max(), name='radius'))
    diameter = pd.DataFrame(df_numeric).groupby(labels).apply(lambda x: pd.Series(x.apply(lambda x: np.linalg.norm(x - cluster_centers.loc[x.name]), axis=1).mean(), name='diameter'))
    density = pd.DataFrame(df_numeric).groupby(labels).apply(lambda x: pd.Series(x.apply(lambda x: np.exp(-(np.linalg.norm(x - cluster_centers.loc[x.name])/radius.loc[x.name])**2), axis=1).mean(), name='density'))
    print(f"k={k}, radius={radius.mean()[0]:.2f}, diameter={diameter.mean()[0]:.2f}, density={density.mean()[0]:.2f}")

# Choose the best k value based on the silhouette score
scores = []
for k in range(8, 17):
    clustering = MiniBatchKMeans(n_clusters=k).fit(features_complete)
    score = silhouette_score(features_complete, clustering.labels_)
    scores.append(score)
best_k = 8 + scores.index(max(scores))

# Cluster the complete dataset using Mini-Batch K-Means with the best k value
clustering = MiniBatchKMeans(n_clusters=best_k).fit(features_complete)


NameError: name 'df_numeric' is not defined